In [7]:
# TODO: make a proper module and remove this hack
import os
import sys
sys.path.append("/home/robbe/open-hardware-definitions/")

from parsers.common import DATASHEET_DIR

from py_pdf_parser import tables
from py_pdf_parser.common import BoundingBox
from py_pdf_parser.loaders import load_file
from py_pdf_parser.visualise import visualise

from collections import namedtuple

from pprint import pprint

In [34]:
Module = namedtuple('Module', ['name', 'base_addr', 'base_addr_str', 'ref_page'])
Register = namedtuple('Register', ['name', 'module', 'addr', 'description', 'read_allowed', 'write_allowed', 'default', 'ref_page'])

In [25]:
# Load with lower char_margin to fix concatenation of columns, and also specify a custom fuzzy orderer

FUZZY_ROUNDING = 5
def fr(val):
    return round(val / FUZZY_ROUNDING) * FUZZY_ROUNDING

orderer = lambda elements: sorted(
    elements, key=lambda elem: (fr(-elem.y0), fr(elem.x0))
)

doc = load_file(os.path.join(DATASHEET_DIR, "MPC5668xRM.pdf"), {'char_margin': 1}, element_ordering=orderer)
content = doc.elements

In [26]:
# Filter out header and footer elements
header_box = BoundingBox(0, 1000, 740, 1000)
footer_box = BoundingBox(0, 1000, 0, 80)
for page in range(1, doc.number_of_pages):
    content -= content.filter_partially_within_bounding_box(header_box, page)
    content -= content.filter_partially_within_bounding_box(footer_box, page)

In [27]:
# Extract module base addresses
mod_base_addr_before = content.filter_by_text_contains("Table A-1. Module Base Addresses")[0]
mod_base_addr_after = content.filter_by_text_contains("Table A-2. MPC5668x System Memory Map")[0]

mod_base_addr_table_elements = content.between(mod_base_addr_before, mod_base_addr_after).filter_by_fonts("IOCJHP+Helvetica-Bold,9.0", "IOCJJB+Helvetica,9.0")
mod_base_addr_table_data = mod_base_addr_table_elements.filter_by_font("IOCJJB+Helvetica,9.0")

for i in mod_base_addr_table_data:
    print(i.text())

t = tables.extract_simple_table(mod_base_addr_table_data, as_text=True)
modules = [Module(d[0], int(d[1].replace("_", ""), 16), d[1], d[2].split("Page ")[1]) for d in t]

for m in modules:
    print(f"Module '{m.name}': Base: {hex(m.base_addr)}, Page: {m.ref_page}")

Program/Data Flash
0x0000_3FFF
Page A-3
Flash Emulation Mapping
0x0100_0000
Page A-3
SRAM
0x4000_0000
Page A-3
MLB_DIM Configuration
0xC3F8_4000
Page A-8
I2C_C
0xC3F8_8000
Page A-11
I2C_D
0xC3F8_C000
Page A-11
DSPI_C
0xC3F9_0000
Page A-12
DSPI_D
0xC3F9_4000
Page A-13
eSCI_J
0xC3FA_0000
Page A-14
eSCI_K
0xC3FA_4000
Page A-14
eSCI_L
0xC3FA_8000
Page A-15
eSCI_M
0xC3FA_C000
Page A-16
FlexRay
0xC3FD_C000
Page A-16
AXBS
0xFFF0_4000
Page A-38
Sema4
0xFFF1_0000
Page A-39
MPU
0xFFF1_4000
Page A-40
SWT
0xFFF3_8000
Page A-42
STM
0xFFF3_C000
Page A-42
ECSM
0xFFF4_0000
Page A-43
eDMA
0xFFF4_4000
Page A-43
INTC
0xFFF4_8000
Page A-46
FEC
0xFFF4_C000
Page A-49
ADC_A
0xFFF8_0000
Page A-51
I2C_A
0xFFF8_8000
Page A-55
I2C_B
0xFFF8_C000
Page A-56
DSPI_A
0xFFF9_0000
Page A-56
DSPI_B
0xFFF9_4000
Page A-57
eSCI_A
0xFFFA_0000
Page A-58
eSCI_B
0xFFFA_4000
Page A-58
eSCI_C
0xFFFA_8000
Page A-59
eSCI_D
0xFFFA_C000
Page A-59
eSCI_E
0xFFFB_0000
Page A-60
eSCI_F
0xFFFB_4000
Page A-60
eSCI_G
0xFFFB_8000
Page A-61
e

In [33]:
# Register map (table A-4)
reg_map_before = content.filter_by_text_contains("Table A-4. MPC5668x Detailed Register Map")[0]
reg_map_after = content.filter_by_text_contains("In this column, R/W indicates a read/write register")[0]
reg_map_elements = content.between(reg_map_before, reg_map_after)

# Find start elements of each module in this table
module_elements = {}
for m in modules:
    tmp_list = reg_map_elements.filter_by_text_contains(m.base_addr_str)
    if len(tmp_list) > 0:
        module_elements[m.name] = tmp_list[0]

# Extract all registers for the module
registers = []
for i, m in enumerate(modules):
    if m.name not in module_elements.keys():
        continue
    if m.name in module_elements.keys() and i < len(modules) - 1:
        tbl_contents = content.between(module_elements[m.name], module_elements[modules[i+1].name]).filter_by_font("IOCJJB+Helvetica,9.0")
    elif m.name in module_elements.keys() and i == len(modules) - 1:
        tbl_contents = content.between(module_elements[m.name], reg_map_after).filter_by_font("IOCJJB+Helvetica,9.0")
    
    if len(tbl_contents) == 0:
        continue
    
    # Get rid of "reserved" lines
    res_elements = tbl_contents.filter_by_text_contains("Reserved")
    for res_element in res_elements:
        tbl_contents -= tbl_contents.horizontally_in_line_with(res_element)
    tbl_contents -= res_elements
    
    # Extract table
    print(m.name)
    tbl = tables.extract_simple_table(tbl_contents, as_text=True, allow_gaps=True)    
    for r in tbl:
        ['name', 'module', 'addr', 'description', 'read_allowed', 'write_allowed', 'default', 'ref_page']
                
        try:
            name = r[1].split('–')[0]
            description = r[1].split('–')[1:]
            addr = int(r[0], 16) + m.base_addr
            read_allowed = 'R' in r[2]
            write_allowed = 'W' in r[2]
            default = r[3] # TODO: clean this up

            registers.append(Register(name, m.name, addr, description, read_allowed, write_allowed, default, r[4]))
        except Ex:
            print(f"CANNOT HANDLE: {r}")
    
    print()
    for r in registers:
        print(r)
        

MLB_DIM Configuration
CANNOT HANDLE: ['0x0000', 'DCCR—Device Control Configuration Register', 'R/W', '0x0000_0000', '26.3.2.1/26-8']
CANNOT HANDLE: ['0x0004', 'SSCR—System Status Configuration Register', 'R/W', '0x0000_0000', '26.3.2.2/26-10']
CANNOT HANDLE: ['0x0008', 'SDCR—System Data Configuration Register', 'R', '0x0000_0000', '26.3.2.3/26-11']
CANNOT HANDLE: ['0x000C', 'SMCR—System Mask Configuration Register', 'R/W', '0x0000_0060', '26.3.2.4/26-12']
CANNOT HANDLE: ['0x001C', 'VCCR—Version Control Configuration Register', 'R', '0x0300_0202', '26.3.2.5/26-13']
CANNOT HANDLE: ['0x0020', 'SBCR—Synchronous Base Address Configuration Register', 'R/W', '0x0000_0000', '26.3.2.6/26-14']
CANNOT HANDLE: ['0x0024', 'ABCR—Asynchronous Base Address Configuration Register', 'R/W', '0x0000_0000', '26.3.2.7/26-14']
CANNOT HANDLE: ['0x0028', 'CBCR—Control Base Address Configuration Register', 'R/W', '0x0000_0000', '26.3.2.8/26-15']
CANNOT HANDLE: ['0x002C', 'IBCR—Isochronous Base Address Configura

CANNOT HANDLE: ['0x0000', 'DSPI_MCR—DSPI module configuration register', 'R/W', '0x0000_4001', '29.3.2.1/29-7']
CANNOT HANDLE: ['0x0008', 'DSPI_TCR—DSPI transfer count register', 'R/W', '0x0000_0000', '29.3.2.2/29-9']
CANNOT HANDLE: ['0x000C', 'DSPI_CTAR0—DSPI clock and transfer attributes register 0', 'R/W', '0x7800_0000', '29.3.2.3/29-10']
CANNOT HANDLE: ['0x0010', 'DSPI_CTAR1—DSPI clock and transfer attributes register 1', 'R/W', '0x7800_0000', '29.3.2.3/29-10']
CANNOT HANDLE: ['0x0014', 'DSPI_CTAR2—DSPI clock and transfer attributes register 2', 'R/W', '0x7800_0000', '29.3.2.3/29-10']
CANNOT HANDLE: ['0x0018', 'DSPI_CTAR3—DSPI clock and transfer attributes register 3', 'R/W', '0x7800_0000', '29.3.2.3/29-10']
CANNOT HANDLE: ['0x001C', 'DSPI_CTAR4—DSPI clock and transfer attributes register 4', 'R/W', '0x7800_0000', '29.3.2.3/29-10']
CANNOT HANDLE: ['0x0020', 'DSPI_CTAR5—DSPI clock and transfer attributes register 5', 'R/W', '0x7800_0000', '29.3.2.3/29-10']
CANNOT HANDLE: ['0x0024', 

CANNOT HANDLE: ['0x0000', 'SEMA4_Gate00—Semaphores gate 0', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0001', 'SEMA4_Gate01—Semaphores gate 1', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0002', 'SEMA4_Gate02—Semaphores gate 2', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0003', 'SEMA4_Gate03—Semaphores gate 3', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0004', 'SEMA4_Gate04—Semaphores gate 4', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0005', 'SEMA4_Gate05—Semaphores gate 5', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0006', 'SEMA4_Gate06—Semaphores gate 6', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0007', 'SEMA4_Gate07—Semaphores gate 7', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0008', 'SEMA4_Gate08—Semaphores gate 8', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x0009', 'SEMA4_Gate09—Semaphores gate 9', 'R/W', '0x00', '14.3.2.1/14-4']
CANNOT HANDLE: ['0x000A', 'SEMA4_Gate10—Semaphores gate 10', 'R/W', '0x00', '14.

CANNOT HANDLE: ['0x0000', 'EDMA_CR—eDMA control register', 'R/W', '0x0000_0400', '23.3.2.1/23-8']
CANNOT HANDLE: ['0x0004', 'EDMA_ESR—eDMA error status register', 'R', '0x0000_0000', '23.3.2.2/23-10']
CANNOT HANDLE: ['0x000C', 'EDMA_ERQRL—eDMA enable request low register\uf020\n(channels 31–00)', 'R/W', '0x0000_0000', '23.3.2.3/23-12']
CANNOT HANDLE: ['0x0014', 'EDMA_EEIRL—eDMA enable error interrupt low register\uf020\n(channels 31–00)', 'R/W', '0x0000_0000', '23.3.2.4/23-13']
CANNOT HANDLE: ['0x0018', 'EDMA_SERQR—eDMA set enable request register', 'W', '0x00', '23.3.2.5/23-14']
CANNOT HANDLE: ['0x0019', 'EDMA_CERQR—eDMA clear enable request register', 'W', '0x00', '23.3.2.6/23-15']
CANNOT HANDLE: ['0x001A', 'EDMA_SEEIR—eDMA set enable error interrupt register', 'W', '0x00', '23.3.2.7/23-15']
CANNOT HANDLE: ['0x001B', 'EDMA_CEEIR—eDMA clear enable error interrupt register', 'W', '0x00', '23.3.2.8/23-16']
CANNOT HANDLE: ['0x001C', 'EDMA_CIRQR—eDMA clear interrupt request register', 'W'

CANNOT HANDLE: ['0x0000', 'INTC_MCR—INTC module configuration register', 'R/W', '0x0000_0000', '9.3.2.1/9-9']
CANNOT HANDLE: ['0x0008', 'INTC_CPR_PRC0—INTC current priority register for \nprocessor 0 (Z6)', 'R/W', '0x0000_000F', '9.3.2.2/9-10']
CANNOT HANDLE: ['0x00C', 'INTC_CPR_PRC1—INTC current priority register for \nprocessor 1 (Z0)', 'R/W', '0x0000_000F', '9.3.2.3/9-11']
CANNOT HANDLE: ['0x0010', 'INTC_IACKR_PRC0—INTC interrupt acknowledge register \nfor processor 0 (Z6)', 'R/W', '0x0000_0000', '9.3.2.4/9-12']
CANNOT HANDLE: ['0x0014', 'INTC_IACKR_PRC1—INTC interrupt acknowledge register \nfor processor 1 (Z0)', 'R/W', '0x0000_0000', '9.3.2.5/9-13']
CANNOT HANDLE: ['0x0018', 'INTC_EOIR_PRC0—INTC end of interrupt register for \nprocessor 0 (Z6)', 'W', '0x0000_0000', '9.3.2.6/9-14']
CANNOT HANDLE: ['0x001C', 'INTC_EOIR_PRC1—INTC end of interrupt register for \nprocessor 1 (Z0)', 'W', '0x0000_0000', '9.3.2.7/9-14']
CANNOT HANDLE: ['0x0020', 'INTC_SSCIR0_3—INTC software set/clear inte

...

CANNOT HANDLE: ['0x0000', 'CAND_MCR—Module configuration register', 'R/W', '0x9890_000F', '28.3.4.1/28-11']
CANNOT HANDLE: ['0x0004', 'CAND_CTRL—control register', 'R/W', '0x0000_0000', '28.3.4.2/28-14']
CANNOT HANDLE: ['0x0008', 'CAND_TIMER—Free-running timer', 'R/W', '0x0000_0000', '28.3.4.3/28-17']
CANNOT HANDLE: ['0x0010', 'CAND_RXGMASK—Rx global mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.1/28-18']
CANNOT HANDLE: ['0x0014', 'CAND_RX14MASK—Rx buffer 14 mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.2/28-19']
CANNOT HANDLE: ['0x0018', 'CAND_RX15MASK—Rx buffer 15 mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.3/28-19']
CANNOT HANDLE: ['0x001C', 'CAND_ECR—Error counter register', 'R/W', '0x0000_0000', '28.3.4.5/28-20']
CANNOT HANDLE: ['0x0020', 'CAND_ESR—Error and status register', 'R/W', '0x0000_0000', '28.3.4.6/28-21']
CANNOT HANDLE: ['0x0024', 'CAND_IMASK2—Interrupt masks 2', 'R/W', '0x0000_0000', '28.3.4.7/28-23']
CANNOT HANDLE: ['0x0028', 'CAND_IMASK1—Interrupt masks 1', 'R/W', '0x0000_0000', '28.3.4

CANNOT HANDLE: ['0x0000', 'CANE_MCR—Module configuration register', 'R/W', '0x9890_000F', '28.3.4.1/28-11']
CANNOT HANDLE: ['0x0004', 'CANE_CTRL—control register', 'R/W', '0x0000_0000', '28.3.4.2/28-14']
CANNOT HANDLE: ['0x0008', 'CANE_TIMER—Free-running timer', 'R/W', '0x0000_0000', '28.3.4.3/28-17']
CANNOT HANDLE: ['0x0010', 'CANE_RXGMASK—Rx global mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.1/28-18']
CANNOT HANDLE: ['0x0014', 'CANE_RX14MASK—Rx buffer 14 mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.2/28-19']
CANNOT HANDLE: ['0x0018', 'CANE_RX15MASK—Rx buffer 15 mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.3/28-19']
CANNOT HANDLE: ['0x001C', 'CANE_ECR—Error counter register', 'R/W', '0x0000_0000', '28.3.4.5/28-20']
CANNOT HANDLE: ['0x0020', 'CANE_ESR—Error and status register', 'R/W', '0x0000_0000', '28.3.4.6/28-21']
CANNOT HANDLE: ['0x0024', 'CANE_IMASK2—Interrupt masks 2', 'R/W', '0x0000_0000', '28.3.4.7/28-23']
CANNOT HANDLE: ['0x0028', 'CANE_IMASK1—Interrupt masks 1', 'R/W', '0x0000_0000', '28.3.4

CANNOT HANDLE: ['0x0000', 'CANF_MCR—Module configuration register', 'R/W', '0x9890_000F', '28.3.4.1/28-11']
CANNOT HANDLE: ['0x0004', 'CANF_CTRL—control register', 'R/W', '0x0000_0000', '28.3.4.2/28-14']
CANNOT HANDLE: ['0x0008', 'CANF_TIMER—Free-running timer', 'R/W', '0x0000_0000', '28.3.4.3/28-17']
CANNOT HANDLE: ['0x0010', 'CANF_RXGMASK—Rx global mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.1/28-18']
CANNOT HANDLE: ['0x0014', 'CANF_RX14MASK—Rx buffer 14 mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.2/28-19']
CANNOT HANDLE: ['0x0018', 'CANF_RX15MASK—Rx buffer 15 mask', 'R/W', '0xFFFF_FFFF', '28.3.4.4.3/28-19']
CANNOT HANDLE: ['0x001C', 'CANF_ECR—Error counter register', 'R/W', '0x0000_0000', '28.3.4.5/28-20']
CANNOT HANDLE: ['0x0020', 'CANF_ESR—Error and status register', 'R/W', '0x0000_0000', '28.3.4.6/28-21']
CANNOT HANDLE: ['0x0024', 'CANF_IMASK2—Interrupt masks 2', 'R/W', '0x0000_0000', '28.3.4.7/28-23']
CANNOT HANDLE: ['0x0028', 'CANF_IMASK1—Interrupt masks 1', 'R/W', '0x0000_0000', '28.3.4

CANNOT HANDLE: ['0x0000', 'CHCONFIG0—Channel #0 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0001', 'CHCONFIG1—Channel #1 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0002', 'CHCONFIG2—Channel #2 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0003', 'CHCONFIG3—Channel #3 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0004', 'CHCONFIG4—Channel #4 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0005', 'CHCONFIG5—Channel #5 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0006', 'CHCONFIG6—Channel #6 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0007', 'CHCONFIG7—Channel #7 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0008', 'CHCONFIG8—Channel #8 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x0009', 'CHCONFIG9—Channel #9 configuration', 'R/W', '0x00', '22.3.2.1/22-4']
CANNOT HANDLE: ['0x000A', 'CHCONFIG10—Ch

CANNOT HANDLE: ['0x0004', 'SIU_MIDR—MCU ID register', 'R', '—5', '7.3.2.1/7-13']
CANNOT HANDLE: ['0x000C', 'SIU_RSR—Reset status register', 'R/W', '0x8000_000U', '7.3.2.2/7-14']
CANNOT HANDLE: ['0x0010', 'SIU_SRCR—System reset control register', 'R/W', '0x0800_C000', '7.3.2.3/7-15']
CANNOT HANDLE: ['0x0014', 'SIU_EISR—SIU External Interrupt Status Register', 'R/W', '0x0000_0000', '7.3.2.4/7-16']
CANNOT HANDLE: ['0x0018', 'SIU_DIRER—DMA/interrupt request enable register', 'R/W', '0x0000_0000', '7.3.2.5/7-17']
CANNOT HANDLE: ['0x001C', 'SIU_DIRSR—DMA/interrupt request select register', 'R/W', '0x0000_0000', '7.3.2.6/7-18']
CANNOT HANDLE: ['0x0020', 'SIU_OSR—Overrun status register', 'R/W', '0x0000_0000', '7.3.2.7/7-19']
CANNOT HANDLE: ['0x0024', 'SIU_ORER—Overrun request enable register', 'R/W', '0x0000_0000', '7.3.2.8/7-19']
CANNOT HANDLE: ['0x0028', 'SIU_IREER—External IRQ rising-edge event enable register', 'R/W', '0x0000_0000', '7.3.2.9/7-20']
CANNOT HANDLE: ['0x002C', 'SIU_IFEER—Ext

In [125]:
# Extract register map
mod_base_addr_before = content.filter_by_text_contains("Table A-4. MPC5668x Detailed Register Map")[0]
mod_base_addr_after = content.filter_by_text_contains("In this column, R/W indicates a read/write register")[0]

mod_base_addr_table_elements = content.between(mod_base_addr_before, mod_base_addr_after).filter_by_fonts("IOCJHP+Helvetica-Bold,9.0", "IOCJJB+Helvetica,9.0")
mod_base_addr_table_data = mod_base_addr_table_elements.filter_by_font("IOCJJB+Helvetica,9.0")

# for e in mod_base_addr_table_data:
#     print(e.text())

import time

st = time.monotonic()

t = tables.extract_table(
        mod_base_addr_table_data,
        as_text=True,
    )

print(f"took {time.monotonic() - st} seconds")

KeyboardInterrupt: 

In [124]:
for l in [(i[0], i[1], i[3]) for i in t]:
    print(l)

('0x4000_0000 – \n0x4007_FFFF', '', 'R/W')
('0x0000', 'DCCR—Device Control Configuration Register', 'R/W 0x0000_0000')
('0x0004', 'SSCR—System Status Configuration Register', 'R/W 0x0000_0000')
('0x0008', 'SDCR—System Data Configuration Register', 'R')
('0x000C', 'SMCR—System Mask Configuration Register', 'R/W 0x0000_0060')
('0x001C', 'VCCR—Version Control Configuration Register', 'R')
('0x0020', 'SBCR—Synchronous Base Address Configuration Register', 'R/W 0x0000_0000')
('0x0024', 'ABCR—Asynchronous Base Address Configuration Register', 'R/W 0x0000_0000')
('0x0028', 'CBCR—Control Base Address Configuration Register', 'R/W 0x0000_0000')
('0x002C', 'IBCR—Isochronous Base Address Configuration Register', 'R/W 0x0000_0000')
('0x0030', 'CICR—Channel Interrupt Configuration Register', 'R')
('0x0040', 'CECR0—Channel 0 Entry Configuration Register', 'R/W 0x0000_0000 26.3.2.11/26-17')
('0x0044', 'CSCR0—Channel 0 Status Configuration Register', 'R/W 0x8000_0000 26.3.2.12/26-19')
('0x0048', 'CCBC